In [12]:
import pandas as pd
from datasets import Dataset, load_dataset
from pandas import DataFrame

# Load Dataset From HuggingFace Hub

In [2]:
dataset = load_dataset("GustavoHCruz/DNA_Coding_Regions", split="train")

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1677609 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [3]:
assert isinstance(dataset, Dataset)

df = dataset.to_pandas()

## Exons and Introns Binary Classification

In [ ]:
dna_df = pd.read_csv("./storage/data/base/dna_sequences.csv", keep_default_na=False)
exin_df = pd.read_csv("./storage/data/base/exin_sequences.csv", keep_default_na=False)

In [ ]:
dna_df = dna_df.fillna("")
exin_df = exin_df.fillna("")

In [ ]:
merged_df = pd.merge(
	dna_df,
	exin_df,
	right_on="dna_accession",
	left_on="accession",
	how="right"
)

In [ ]:
filtered_df = merged_df[merged_df["sequence_y"].str.len() < 950]

filtered_df = filtered_df.rename(columns={
	"sequence_y": "sequence",
	"type": "target"
})[["sequence", "target", "organism", "gene", "before", "after"]]

filtered_df.to_csv("./storage/data/processed/exin-950.csv", index=False)

In [ ]:
filtered_df = merged_df[merged_df["sequence_y"].str.len() < 512]

filtered_df = filtered_df.rename(columns={
	"sequence_y": "sequence",
	"type": "target"
})[["sequence", "target", "organism", "gene", "before", "after"]]

filtered_df.to_csv("./storage/data/processed/exin-512.csv", index=False)

In [ ]:
filtered_df = merged_df[merged_df["sequence_y"].str.len() < 256]

filtered_df = filtered_df.rename(columns={
	"sequence_y": "sequence",
	"type": "target"
})[["sequence", "target", "organism", "gene", "before", "after"]]

filtered_df.to_csv("./storage/data/processed/exin-256.csv", index=False)

## Nucleotide Classification

In [ ]:
dna_df = pd.read_csv("./storage/data/base/dna_sequences.csv", keep_default_na=False)
exin_df = pd.read_csv("./storage/data/base/exin_sequences.csv", keep_default_na=False)

In [ ]:
merged_df = pd.merge(
	dna_df,
	exin_df,
	right_on="dna_accession",
	left_on="accession",
	how="right"
)

In [ ]:
def build_target(
	seq: str,
	exins: DataFrame
) -> str:
	target = np.array(["U"] * len(seq))

	for _, exin in exins.iterrows():
		start, end, etype = int(exin["start"]), int(exin["end"]), exin["type"]
		if etype.lower().startswith("exon"):
			target[start:end] = "E"
		elif etype.lower().startswith("intron"):
			target[start:end] = "I"

	return "".join(target)

In [ ]:
filtered_df = merged_df[merged_df["sequence_x"].str.len() < 2000]

results = []
for accession, group in filtered_df.groupby("accession"):
	seq = group.iloc[0]["sequence_x"]
	organism = group.iloc[0]["organism"]
	target = build_target(seq, group)
	results.append({
		"sequence": seq,
		"target": target,
		"organism": organism
	})

result_df = pd.DataFrame(results)

result_df.to_csv("./storage/data/processed/nucl-2000.csv")

In [ ]:
filtered_df = merged_df[merged_df["sequence_x"].str.len() < 1000]

results = []
for accession, group in filtered_df.groupby("accession"):
	seq = group.iloc[0]["sequence_x"]
	organism = group.iloc[0]["organism"]
	target = build_target(seq, group)
	results.append({
		"sequence": seq,
		"target": target,
		"organism": organism
	})

result_df = pd.DataFrame(results)

result_df.to_csv("./storage/data/processed/nucl-1000.csv")

In [ ]:
filtered_df = merged_df[merged_df["sequence_x"].str.len() < 500]

results = []
for accession, group in filtered_df.groupby("accession"):
	seq = group.iloc[0]["sequence_x"]
	organism = group.iloc[0]["organism"]
	target = build_target(seq, group)
	results.append({
		"sequence": seq,
		"target": target,
		"organism": organism
	})

result_df = pd.DataFrame(results)

result_df.to_csv("./storage/data/processed/nucl-500.csv")

## DNA Translator

In [29]:
def join_proteins(protein_list) -> str:
	seqs = []
	for p in protein_list:
		seq = p.get("sequence", "")
		if isinstance(seq, str):
			seqs.append(seq)
	return "".join(seqs)

In [32]:
assert isinstance(df, DataFrame)

df["target"] = df["proteins"].apply(join_proteins).astype(str)
mask = (df["sequence"].str.len() + df["target"].str.len()) < 500
filtered_df = df[mask]

final_df = filtered_df[["sequence", "target", "organism"]]

final_df.to_csv("./storage/data/processed/tran-500.csv", index=False)

In [33]:
assert isinstance(df, DataFrame)

df["target"] = df["proteins"].apply(join_proteins).astype(str)
mask = (df["sequence"].str.len() + df["target"].str.len()) < 1000
filtered_df = df[mask]

final_df = filtered_df[["sequence", "target", "organism"]]

final_df.to_csv("./storage/data/processed/tran-1000.csv", index=False)